In [31]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [39]:
train_datagen = ImageDataGenerator(
    rescale = 1/.255,
    validation_split = 0.2
)

train_data = train_datagen.flow_from_directory('./rice-data/',
                                                target_size=(128, 128),
                                                color_mode='grayscale',
                                                class_mode='categorical',
                                                batch_size = 256,
                                                subset = 'training')

valid_data = train_datagen.flow_from_directory('./rice-data/',
                                                target_size=(128, 128),
                                                color_mode='grayscale',
                                                class_mode='categorical',
                                                batch_size = 256,
                                                subset = 'validation')

test = train_datagen.flow_from_directory('./rice-data/',
                                                target_size=(128, 128),
                                                color_mode='grayscale',
                                                class_mode='categorical',
                                                batch_size = 256,
                                                subset = 'validation')

Found 60000 images belonging to 5 classes.
Found 15000 images belonging to 5 classes.
Found 15000 images belonging to 5 classes.


In [33]:
train_data.class_indices

{'Arborio': 0, 'Basmati': 1, 'Ipsala': 2, 'Jasmine': 3, 'Karacadag': 4}

In [34]:
import keras
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten
from keras.models import Sequential

In [35]:
model = Sequential()
model.add(Conv2D(32, (3,3), activation = 'relu', input_shape = (128,128,1)))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(128, (3,3), activation = 'relu'))
model.add(MaxPooling2D(2,2))
model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_14 (Conv2D)              │ (None, 126, 126, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_14 (MaxPooling2D) │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_15 (Conv2D)              │ (None, 61, 61, 128)    │        36,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_15 (MaxPooling2D) │ (None, 30, 30, 128)    │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 37,312 (145.75 KB)

 Trainable params: 37,312 (145.75 KB)

 Non-trainable params: 0 (0.00 B)

In [36]:
model.add(Flatten())
model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.3))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(5, activation = 'sigmoid'))
model.compile(loss = 'categorical_crossentropy', metrics = ['accuracy'], optimizer='adam')
model.fit(train_data, validation_data= valid_data, epochs=5)

Epoch 1/5
235/235 ━━━━━━━━━━━━━━━━━━━━ 474s 2s/step - accuracy: 0.7848 - loss: 34.9296 - val_accuracy: 0.9685 - val_loss: 0.0926
Epoch 2/5
235/235 ━━━━━━━━━━━━━━━━━━━━ 509s 2s/step - accuracy: 0.9624 - loss: 0.1202 - val_accuracy: 0.9703 - val_loss: 0.0920
Epoch 3/5
235/235 ━━━━━━━━━━━━━━━━━━━━ 525s 2s/step - accuracy: 0.9690 - loss: 0.0933 - val_accuracy: 0.9709 - val_loss: 0.0866
Epoch 4/5
235/235 ━━━━━━━━━━━━━━━━━━━━ 559s 2s/step - accuracy: 0.9711 - loss: 0.0847 - val_accuracy: 0.9717 - val_loss: 0.0885
Epoch 5/5
235/235 ━━━━━━━━━━━━━━━━━━━━ 545s 2s/step - accuracy: 0.9776 - loss: 0.0648 - val_accuracy: 0.9742 - val_loss: 0.0816


In [37]:
model.save('rice-class.h5')

In [40]:
model.evaluate(test)

/Users/bhagya/Desktop/code/.venv/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


59/59 ━━━━━━━━━━━━━━━━━━━━ 25s 419ms/step - accuracy: 0.9722 - loss: 0.0890


[0.0815778374671936, 0.9742000102996826]